# Import Necessary Files

In [1]:
import keras
import tensorflow as tf

In [2]:
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D
from keras.layers import BatchNormalization, Activation, MaxPool2D
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import RMSprop
from keras.models import Model, load_model
from keras.metrics import Accuracy
from keras import utils
import imutils
import numpy as np

# Data Augmentation

In [3]:
from keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array, load_img
import matplotlib

train_datagen = ImageDataGenerator(rotation_range=40,
                                  width_shift_range=.2,
                                  height_shift_range=.2,
                                  rescale=1./255,
                                  zoom_range=.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)


### Read Train dir of Images

In [4]:
batch_size = 10

train_data = train_datagen.flow_from_directory(r'D:\Courses\LetsUpgrade\Project 10 - Mask detection using OpenCV\observations-master\experiements\dest_folder\train',
                                              target_size=(100,100),batch_size=batch_size,
                                              class_mode='binary')


Found 1315 images belonging to 2 classes.


### Read Test dir Images

In [5]:
tes_data = test_datagen.flow_from_directory(r'D:\Courses\LetsUpgrade\Project 10 - Mask detection using OpenCV\observations-master\experiements\dest_folder\test',
                                           target_size = (100,100),
                                           batch_size = batch_size,
                                           class_mode = 'binary')

Found 194 images belonging to 2 classes.


### CNN Model

In [15]:
model = Sequential([
    Conv2D(100,(3,3),activation='relu',input_shape=(100,100,3)),
    MaxPool2D(2,2),
    
    Conv2D(100,(3,3),activation='relu'),
    MaxPool2D(2,2),
    
    Conv2D(100,(3,3),activation='relu'),
    MaxPool2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50,activation='relu'),
    
    Dense(2,activation='softmax')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 98, 98, 100)       2800      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 49, 49, 100)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 47, 47, 100)       90100     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 23, 23, 100)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 21, 21, 100)       90100     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 10, 10, 100)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 10000)            

### Complilation of CNN

In [16]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Train the model with the Data

In [19]:
history = model.fit_generator(train_data,steps_per_epoch=50//batch_size,
                             epochs=10,
                             validation_data=tes_data,
                             validation_steps=20//batch_size)

Epoch 1/10
5/5 [==============================] - 1s 185ms/step - loss: 0.6932 - accuracy: 0.3600 - val_loss: 0.6932 - val_accuracy: 0.4000
Epoch 2/10
5/5 [==============================] - 1s 174ms/step - loss: 0.6932 - accuracy: 0.5600 - val_loss: 0.6932 - val_accuracy: 0.6500
Epoch 3/10
5/5 [==============================] - 1s 204ms/step - loss: 0.6932 - accuracy: 0.4400 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/10
5/5 [==============================] - 1s 175ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/10
5/5 [==============================] - 1s 224ms/step - loss: 0.6932 - accuracy: 0.4600 - val_loss: 0.6932 - val_accuracy: 0.4500
Epoch 6/10
5/5 [==============================] - 1s 184ms/step - loss: 0.6932 - accuracy: 0.5800 - val_loss: 0.6932 - val_accuracy: 0.5500
Epoch 7/10
5/5 [==============================] - 1s 193ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.6000
Epoch 8/10
5/5 [====

#### Save trained model

In [20]:
model.save('detection.h5')

## Implement Face Mask Detection with opencv - Haarcascade Classifier

In [ ]:
model = tf.keras.models.load_model('detection.h5')
results = {0:'without mask', 1:'with mask'}
GR_dict={0:(0,0,255),1:(0,0,255)}         ### Group dictionary

rect_size=4

cap=cv2.VideoCapture(0)
haarcascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(100,100))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,100,100,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('LIVE',im)
    key = cv2.waitKey(10)
    
    if key == 'q': 
        break

cap.release()

cv2.destroyAllWindows()